<a href="https://colab.research.google.com/github/B0829009/pa/blob/main/dose_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#connect to google drive

In [ ]:
#@title
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#load package from google drive

In [ ]:
#@title
%reload_ext rpy2.ipython

In [ ]:
#@title
%%R
lib_loc <- "/content/drive/MyDrive/Colab_Notebooks/r_lib"
lib_loc

[1] "/content/drive/MyDrive/Colab_Notebooks/r_lib"


In [ ]:
#@title
%%R
library(DOSE, lib=lib_loc)
library(enrichplot, lib=lib_loc)
library(clusterProfiler, lib=lib_loc)
library(cowplot, lib=lib_loc)

R[write to console]: 

R[write to console]: DOSE v3.18.3  For help: https://guangchuangyu.github.io/software/DOSE

If you use DOSE in published research, please cite:
Guangchuang Yu, Li-Gen Wang, Guang-Rong Yan, Qing-Yu He. DOSE: an R/Bioconductor package for Disease Ontology Semantic and Enrichment analysis. Bioinformatics 2015, 31(4):608-609


R[write to console]: clusterProfiler v4.0.5  For help: https://yulab-smu.top/biomedical-knowledge-mining-book/

If you use clusterProfiler in published research, please cite:
T Wu, E Hu, S Xu, M Chen, P Guo, Z Dai, T Feng, L Zhou, W Tang, L Zhan, X Fu, S Liu, X Bo, and G Yu. clusterProfiler 4.0: A universal enrichment tool for interpreting omics data. The Innovation. 2021, 2(3):100141. doi: 10.1016/j.xinn.2021.100141

R[write to console]: 
Attaching package: ‘clusterProfiler’


R[write to console]: The following object is masked from ‘package:stats’:

    filter




In [ ]:
#@title
%%R  
library(BiocGenerics, lib=lib_loc) 
library(S4Vectors, lib=lib_loc) #TRanges
library(IRanges, lib=lib_loc) #AnnotationDbi
library(Biobase, lib=lib_loc) 
library(AnnotationDbi, lib=lib_loc)
library(org.Hs.eg.db, lib=lib_loc)

R[write to console]: Loading required package: parallel

R[write to console]: 
Attaching package: ‘BiocGenerics’


R[write to console]: The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


R[write to console]: The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


R[write to console]: The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


R[write to console]: Loading required package: sta

In [ ]:
#@title
%%R
library(ggnewscale, lib=lib_loc)
library(ggupset, lib=lib_loc)
library(ggridges, lib=lib_loc)
library(europepmc, lib=lib_loc)
library(ggplot2)
library(httr)
library(dplyr) 

R[write to console]: 
Attaching package: ‘httr’


R[write to console]: The following object is masked from ‘package:Biobase’:

    content


R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following object is masked from ‘package:AnnotationDbi’:

    select


R[write to console]: The following object is masked from ‘package:Biobase’:

    combine


R[write to console]: The following objects are masked from ‘package:IRanges’:

    collapse, desc, intersect, setdiff, slice, union


R[write to console]: The following objects are masked from ‘package:S4Vectors’:

    first, intersect, rename, setdiff, setequal, union


R[write to console]: The following objects are masked from ‘package:BiocGenerics’:

    combine, intersect, setdiff, union


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




#選檔案


In [ ]:
#上傳檔案到colab
from google.colab import files
from rpy2.robjects import r
import rpy2.robjects as robjects

file=files.upload()

if (file):
  filename = next(iter(file))
  r = robjects.r
  r.assign('filename', filename) #將檔案名稱python -> R
else:
  print("請再次執行此儲存格，並選擇一個檔案")

Saving new_test.csv to new_test.csv


#data

In [ ]:
%%R
filetype <- sub(".*(txt).*", replacement = "\\1", filename)
filetype <- sub(".*(csv).*", replacement = "\\1", filename)
print(filetype)
if (filetype == "txt")
  d = read.txt(file = filename, header=TRUE ,sep="\t" ,fileEncoding ="UTF-8")
else if (filetype == "csv")
  d = read.csv(file = filename, header=TRUE, fileEncoding ="UTF-8")
else
  print("檔案格式錯誤 請再試一遍")

print(dim(d))
print(colnames(d))
d[,'ID'] <- sub("-.*", "", d$ID)
#write.table(d[,'ID'], file="sub.csv", quote = FALSE, row.names = F)
print(head(d))

[1] "csv"
[1] 3749   18
 [1] "name"                   "DMSO_1"                 "DMSO_2"                
 [4] "DMSO_3"                 "DMSO_4"                 "LA_1"                  
 [7] "LA_2"                   "LA_3"                   "LA_4"                  
[10] "ID"                     "LA_vs_DMSO_p.val"       "LA_vs_DMSO_p.adj"      
[13] "significant"            "LA_vs_DMSO_significant" "LA_vs_DMSO_ratio"      
[16] "DMSO_centered"          "LA_centered"            "GeneID"                
   name   DMSO_1   DMSO_2   DMSO_3   DMSO_4     LA_1     LA_2     LA_3     LA_4
1 Ifit3 23.82695 24.04958 24.01282 24.14250 25.23222 25.24583 25.17408 25.10332
2 Tgtp1 18.92257 18.15192 18.58612 18.95540 19.94380 20.01441 19.69667 19.66601
3 Oas1a 24.27161 24.37087 24.21992 24.34267 25.34469 25.40400 25.27210 25.24794
4   Fyn 21.14772 20.86115 21.20696 20.87048 22.08221 22.43781 21.29188       NA
5 Iigp1 24.48472 24.54531 24.58129 24.63386 25.37996 25.53448 25.27284 25.33561
6  Zbp1 21.13596

In [ ]:
#舊的 先不用跑

%%R
#d = read.csv('/content/drive/MyDrive/Colab_Notebooks/new_test.csv',header=TRUE , sep="\t",fileEncoding ="UTF-8") #選擇檔案  file.choose()
d = read.csv(file=filename,header=TRUE ,fileEncoding ="UTF-8") #選擇檔案  file.choose()
#print(head(d))
print(dim(d))
print(colnames(d))
d[,'ID'] <- sub("-.*", "", d$ID)
#write.table(d[,'ID'], file="sub.csv", quote = FALSE, row.names = F)
print(head(d))

[1] 3749   18
 [1] "name"                   "DMSO_1"                 "DMSO_2"                
 [4] "DMSO_3"                 "DMSO_4"                 "LA_1"                  
 [7] "LA_2"                   "LA_3"                   "LA_4"                  
[10] "ID"                     "LA_vs_DMSO_p.val"       "LA_vs_DMSO_p.adj"      
[13] "significant"            "LA_vs_DMSO_significant" "LA_vs_DMSO_ratio"      
[16] "DMSO_centered"          "LA_centered"            "GeneID"                
   name   DMSO_1   DMSO_2   DMSO_3   DMSO_4     LA_1     LA_2     LA_3     LA_4
1 Ifit3 23.82695 24.04958 24.01282 24.14250 25.23222 25.24583 25.17408 25.10332
2 Tgtp1 18.92257 18.15192 18.58612 18.95540 19.94380 20.01441 19.69667 19.66601
3 Oas1a 24.27161 24.37087 24.21992 24.34267 25.34469 25.40400 25.27210 25.24794
4   Fyn 21.14772 20.86115 21.20696 20.87048 22.08221 22.43781 21.29188       NA
5 Iigp1 24.48472 24.54531 24.58129 24.63386 25.37996 25.53448 25.27284 25.33561
6  Zbp1 21.13596 20.96163 

In [ ]:
%%R 

#連接網頁(uniprot -> entrez)
ID_string <- paste(d$ID, collapse=' ')

#送出資料
library(httr)
result_1 <- httr::POST("https://www.uniprot.org/uploadlists/", body = list('uploadQuery'=ID_string , 'from'="ACC,ID",'to'="P_ENTREZGENEID")) #c("C0HK80 C0HKD9")
print(paste("result: ",result_1[1]))

#得到轉換結果
table_url <- paste0(result_1[1], ".tab")
result <- httr::GET(table_url)
print(paste("result table: ", result[1]))

#把得到的結果轉成table

result_table <- content(result)
uniprot_entrez <- read.table(text=result_table, col.names=c('uniprot', 'entrez'))
uniprot_entrez <- uniprot_entrez[-c(1),]

#去除重複
uniprot_entrez[, 2] <- as.numeric(uniprot_entrez[, 2])
uniprot_entrez <-uniprot_entrez [ order(uniprot_entrez$uniprot, uniprot_entrez$entrez),]
uniprot_entrez <- uniprot_entrez[!duplicated(uniprot_entrez$uniprot),]

cat("row * colmun =",dim(uniprot_entrez), "\n")
print(head(uniprot_entrez))
write.table(uniprot_entrez, file="uniprot_entrez.csv", quote=FALSE, row.names=F)

[1] "result:  https://www.uniprot.org/mapping/M202111114ABAA9BC7178C81CEBC9459510EDDEA32D096FQ"
[1] "result table:  https://www.uniprot.org/mapping/M202111114ABAA9BC7178C81CEBC9459510EDDEA32D096FQ.tab"


R[write to console]: No encoding supplied: defaulting to UTF-8.



row * colmun = 3602 2 
     uniprot entrez
849   A2A432  72584
3643  A2A5N8 241764
2475  A2A5R2  99371
975   A2A8U2 230917
1705  A2A8Z1 100273
3390  A2ADY9  68817


biomaRt


In [ ]:
%%R
#老鼠(entrezid) => 人類(entrez)
library(biomaRt, lib=lib_loc)

mouse = useEnsembl("ensembl", dataset = "mmusculus_gene_ensembl")
human = useEnsembl("ensembl", dataset = "hsapiens_gene_ensembl")

human_mouse <- getLDS(mart=mouse, attributes=c("entrezgene_id"), filters="entrezgene_id" , values=uniprot_entrez[,"entrez"], attributesL= c("entrezgene_id"), martL = human, uniqueRows=T ) #values=uniprots_ID



In [ ]:
%%R
#處理重複
colnames(human_mouse) <- c('entrez','human')
human_mouse <-human_mouse[ order(human_mouse$entrez, human_mouse$human),]
human_mouse <- human_mouse[!duplicated(human_mouse$entrez),]

print(dim(human_mouse))
print(head(human_mouse))

[1] 3443    2
     entrez human
1818  11306    22
1290  11308 10006
3128  11363    33
3514  11364    34
2931  11370    37
1245  11409    35


In [ ]:
%%R 
#inner join d, human_mouse,uniprot_entrez 
library(dplyr) 

#temp:entrez(mmu)、human、ID(uniprot)
temp <- inner_join( uniprot_entrez, human_mouse, by="entrez")
colnames(temp) <- c('ID', 'entrez', 'human')
temp[, "ID"] <- as.character(temp[, "ID"])
head(temp)

#data
data <- inner_join(temp, d, by="ID")

print(head(data))
cat("uniprot_entrez:",dim(uniprot_entrez), "\n") #老鼠uniprot、老鼠entrez
cat("human_mouse:",dim(human_mouse), "\n") #老鼠entrez、人類entrez
cat("temp", dim(temp), "\n")#老鼠uniprot、老鼠entrez、人類entrez
cat("data:",dim(data), "\n")

#write.table(uniprot_entrez, file="data.csv", quote=FALSE, row.name=F)
#write.table(human_mouse, file="data.csv", quote=FALSE, row.name=F)
#write.table(data, file="data.csv", quote=FALSE, row.name=F)

      ID entrez  human    name   DMSO_1   DMSO_2   DMSO_3   DMSO_4     LA_1
1 A2A5R2  99371  10564 Arfgef2 23.29912 22.79016 23.03723 23.18972 23.01623
2 A2A8U2 230917 199953 Tmem201 21.95978 22.21181 22.00432 22.06360 22.07601
3 A2A8Z1 100273 114883  Osbpl9 24.08747 24.15260 24.17161 23.98436 24.14774
4 A2ADY9  68817   6248    Ddi2 22.84862 22.72420 22.93821 22.73486 22.52323
5 A2AF47  75974 139818  Dock11 21.31911 21.09066 21.24871 21.34867 21.15521
6 A2AGT5  75786   9793   Ckap5 24.16800 24.25111 24.22847 24.20439 24.18573
      LA_2     LA_3     LA_4 LA_vs_DMSO_p.val LA_vs_DMSO_p.adj significant
1 23.00096 23.06632 22.98203      0.545154493            0.973       FALSE
2 22.29341 21.99556 22.19818      0.331773456            0.967       FALSE
3 23.81327 24.26533 24.21237      0.915954050            0.978       FALSE
4 22.66537 22.57378 22.60040      0.004813982            0.753       FALSE
5 21.22370 21.15882 21.13767      0.198564857            0.959       FALSE
6 24.30598 24.3321

In [ ]:
%%R
#geneList
print(colnames(data))
## feature 1: numeric vector
geneList = data[,'LA_vs_DMSO_ratio'] #-log(pValue)

## feature 2: named vector
names(geneList) = as.character(data[, 'human']) 

## feature 3: decreasing orde
geneList = sort(geneList, decreasing = TRUE)
head(geneList)


 [1] "ID"                     "entrez"                 "human"                 
 [4] "name"                   "DMSO_1"                 "DMSO_2"                
 [7] "DMSO_3"                 "DMSO_4"                 "LA_1"                  
[10] "LA_2"                   "LA_3"                   "LA_4"                  
[13] "LA_vs_DMSO_p.val"       "LA_vs_DMSO_p.adj"       "significant"           
[16] "LA_vs_DMSO_significant" "LA_vs_DMSO_ratio"       "DMSO_centered"         
[19] "LA_centered"            "GeneID"                
  3437   4938   2534  81030 439996   9636 
 1.180  1.020  0.964  0.817  0.802  0.798 


In [ ]:
%%R
de <- names(geneList)[abs(log(geneList)) > 1]
edo <- enrichDGN(de)
edo2 <- gseDO(geneList, pvalueCutoff=1)
edox <- setReadable(edo, 'org.Hs.eg.db', 'ENTREZID')

R[write to console]: preparing geneSet collections...

R[write to console]: GSEA analysis...

R[write to console]: leading edge analysis...

R[write to console]: done...



#圖

1.Bar Plot

In [ ]:
%%R
barplot_1 <- function(){
    pic1 <- barplot(edo,x = "Count", color="p.adjust", showCategory=20)+ xlab("Count")
    print(pic1)
    save_plot("pic1.png", pic1, base_height = 10, base_aspect_ratio = 1)
}

2.Dot Plot

In [ ]:
%%R
dotplot_2 <- function(){
   pic2_1 <- dotplot(edo, showCategory=30) + ggtitle("dotplot for ORA")
   pic2_2 <- dotplot(edo2, showCategory=30) + ggtitle("dotplot for GSEA")
   print(pic2_1)
   print(pic2_2)
   save_plot("pic2_1.png", pic2_1, base_height = 10, base_aspect_ratio = 1)
   save_plot("pic2_2.png", pic2_2, base_height = 10, base_aspect_ratio = 1)
}


 3.Gene-Concept Network

In [ ]:
%%R 
gene_concept_network_3 <- function(){
    pic3_1 <- cnetplot(edox, foldChange=geneList)
    ## categorySize can be scaled by 'pvalue' or 'geneNum'
    pic3_2 <- cnetplot(edox, categorySize="pvalue", foldChange=geneList)
    pic3_3 <- cnetplot(edox, foldChange=geneList, circular = TRUE, colorEdge = TRUE)
    print(pic3_1)
    print(pic3_2)
    print(pic3_3)
    pic3_4 <- cnetplot(edox, node_label="category") 
    pic3_5 <- cnetplot(edox, node_label="gene") 
    pic3_6 <- cnetplot(edox, node_label="all") 
    pic3_7 <- cnetplot(edox, node_label="none") 
    print(pic3_4)
    print(pic3_5)
    print(pic3_6)
    print(pic3_7)
    save_plot("pic3_1.png", pic3_1, base_height = 10, base_aspect_ratio = 1)
    save_plot("pic3_2.png", pic3_2, base_height = 10, base_aspect_ratio = 1)
    save_plot("pic3_3.png", pic3_3, base_height = 10, base_aspect_ratio = 1)
    save_plot("pic3_4.png", pic3_4, base_height = 10, base_aspect_ratio = 1)
    save_plot("pic3_5.png", pic3_5, base_height = 10, base_aspect_ratio = 1)
    save_plot("pic3_6.png", pic3_6, base_height = 10, base_aspect_ratio = 1)
    save_plot("pic3_7.png", pic3_7, base_height = 10, base_aspect_ratio = 1)
}

4.Heatmap-like functional classification

In [ ]:
%%R
heatmap_like_functional_classification_4 <- function(){
    pic4_1 <- heatplot(edox)
    pic4_2 <- heatplot(edox, foldChange=geneList)
    cowplot::plot_grid(pic4_1, pic4_2, ncol=1, labels=LETTERS[1:2])
    pic4_1 <- pic4_1 + coord_flip()
    pic4_2 <- pic4_2 + coord_flip()
    print(pic4_1)
    print(pic4_2)
    save_plot("pic4_1.png", pic4_1, base_height = 70, base_aspect_ratio = 0.5,limitsize = FALSE)
    save_plot("pic4_2.png", pic4_2, base_height = 10, base_aspect_ratio = 3,limitsize = FALSE)
}

5 Enrichment Map

In [ ]:
%%R
enrichment_map_5 <- function(){
  edo <- pairwise_termsim(edo)
  pic5_1 <- emapplot(edo)
  pic5_2 <- emapplot(edo, cex_category=1.5)
  pic5_3 <- emapplot(edo, layout="kk")
  pic5_4 <- emapplot(edo, cex_category=1.5,layout="kk") 
  enrichment <- cowplot::plot_grid(pic5_1, pic5_2, pic5_3, pic5_4,labels=LETTERS[1:4])
  print(pic5_1)
  print(pic5_2)
  print(pic5_3)
  print(pic5_4) 
  save_plot("pic5_1.png", pic5_1, base_height = 10, base_aspect_ratio = 1)
  save_plot("pic5_2.png", pic5_2, base_height = 10, base_aspect_ratio = 1)
  save_plot("pic5_3.png", pic5_3, base_height = 10, base_aspect_ratio = 1)
  save_plot("pic5_4.png", pic5_4, base_height = 10, base_aspect_ratio = 1) 
}


6 Biological theme comparison

In [ ]:
%%R
biological_theme_comparison_6 <- function() {
  library(clusterProfiler, lib=lib_loc)
  xx <- compareCluster(data, fun="enrichKEGG",
            organism="hsa", pvalueCutoff=0.05) #pvalueCutoff正常為0.05
  xx <- pairwise_termsim(xx)                     
  pic6_1 <- emapplot(xx)
  pic6_2 <- emapplot(xx, legend_n=2) 
  pic6_3 <- emapplot(xx, pie="count")
  pic6_4 <- emapplot(xx, pie="count", cex_category=1.5, layout="kk")
  #cowplot::plot_grid(p1, p2, p3, p4, ncol=2, labels=LETTERS[1:4])
  print(pic6_1)
  print(pic6_2)
  print(pic6_3)
  print(pic6_4)
  save_plot("pic6_1.png", pic6_1, base_height = 10, base_aspect_ratio = 1)
  save_plot("pic6_2.png", pic6_2, base_height = 10, base_aspect_ratio = 1)
  save_plot("pic6_3.png", pic6_3, base_height = 10, base_aspect_ratio = 1)
  save_plot("pic6_4.png", pic6_4, base_height = 10, base_aspect_ratio = 1)
}


7 UpSet Plot

In [ ]:
%%R
upSet_plot_7 <- function() {
    pic7_1 <- upsetplot(edo)

    kk2 <- gseKEGG(   geneList = geneList,
               organism     = 'hsa',
               minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
    pic7_2 <- upsetplot(kk2, n = 10)
    
    print(pic7_1)
    print(pic7_2)
    save_plot("pic7_1.png", pic7_1, base_height = 10, base_aspect_ratio = 1.5)
    save_plot("pic7_2.png", pic7_2, base_height = 10, base_aspect_ratio = 1.5)

}

8 ridgeline plot for expression distribution of GSEA result (X軸)

In [ ]:
%%R
ridgeline_plot_for_expression_distribution_of_GSEA_result_8 <- function(){
    pic8 <- ridgeplot(edo2) + xlab("expression distributions of enriched genes (log2FC)")
    print(pic8)
    save_plot("pic8.pdf", pic8, base_height = 20, base_aspect_ratio = 0.6)
}

9 running score and preranked list of GSEA result

In [ ]:
%%R
running_score_and_preranked_list_of_GSEA_result_9 <- function(){
  pic9_1 <- gseaplot(edo2, geneSetID = 1, by = "runningScore", title = edo2$Description[1])
  pic9_2 <- gseaplot(edo2, geneSetID = 1, by = "preranked", title = edo2$Description[1])
  pic9_3 <- gseaplot(edo2, geneSetID = 1, title = edo2$Description[1])
  
  #cowplot::plot_grid(pic9_1, pic9_2, pic9_3, ncol=1, labels=LETTERS[1:3])
  pic9_4 <- gseaplot2(edo2, geneSetID = 1, title = edo2$Description[1])
 
  pic9_5 <- gseaplot2(edo2, geneSetID = 1:3)
  pic9_6 <- gseaplot2(edo2, geneSetID = 1:3, pvalue_table = TRUE,
          color = c("#E495A5", "#86B875", "#7DB0DD"), ES_geom = "dot")
  
  pic9_7 <- gseaplot2(edo2, geneSetID = 1:3, subplots = 1) + xlab("Rank")
  pic9_8 <- gseaplot2(edo2, geneSetID = 1:3, subplots = 1:2) + xlab("Rank")
  #pic9_7_8 <- cowplot::plot_grid(pic9_7, pic9_8, ncol=1, labels=LETTERS[1:2])

  pic9_9 <- gsearank(edo2, 1, title = edo2[1, "Description"])

  pic9_10 <- lapply(1:3, function(i){
    anno <- edo2[i, c("NES", "pvalue", "p.adjust")]
    lab <- paste0(names(anno), "=",  round(anno, 3), collapse="\n")
    gsearank(edo2, i, edo2[i, 2]) + xlab("Rank")+ylab("???")+ 
      annotate("text", 5000, edo2[i, "enrichmentScore"]* .75, label =lab, hjust=0, vjust=0)})

#plot_grid(plotlist=pp, ncol=1)
  print(pic9_1)
  print(pic9_2)
  print(pic9_3)
  print(pic9_4)
  print(pic9_5)
  print(pic9_6)
  print(pic9_7)
  print(pic9_8)
  print(pic9_9)
  print(pic9_10)
  save_plot("pic9_1.png", pic9_1, base_height = 10, base_aspect_ratio = 1)
  save_plot("pic9_2.png", pic9_2, base_height = 10, base_aspect_ratio = 1)
  save_plot("pic9_3.png", pic9_3, base_height = 10, base_aspect_ratio = 1)
  save_plot("pic9_4.png", pic9_4, base_height = 10, base_aspect_ratio = 1)
  save_plot("pic9_5.png", pic9_5, base_height = 10, base_aspect_ratio = 1)
  save_plot("pic9_6.png", pic9_6, base_height = 10, base_aspect_ratio = 1)
  save_plot("pic9_7.png", pic9_7, base_height = 10, base_aspect_ratio = 1)
  save_plot("pic9_8.png", pic9_8, base_height = 10, base_aspect_ratio = 1)
  save_plot("pic9_9.png", pic9_9, base_height = 10, base_aspect_ratio = 1)
  #save_plot("pic9_10.png", pic9_10, base_height = 10, base_aspect_ratio = 1)
  
}

#開始畫圖

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
output = widgets.Output()

@output.capture(clear_output=True,wait=True) # based on https://github.com/jupyter-widgets/ipywidgets/issues/1846 and https://ipywidgets.readthedocs.io/en/latest/examples/Output%20Widget.html
def draw(b):
   if (filename):
      
      print ("1. barplot")
      %R barplot_1()
      print ("2. dotplot")
      %R dotplot_2()
      print ("3. Gene-Concept Network")
      %R gene_concept_network_3()
      print ("4. Heatmap-like functional classification")
      %R heatmap_like_functional_classification_4()
      print ("5. Enrichment Map")
      %R enrichment_map_5()
      print ("6. Biological theme comparison")
      %R biological_theme_comparison_6()
      print ("7. UpSet Plot")
      %R upSet_plot_7()
      print ("8. ridgeline plot for expression distribution of GSEA result")
      %R ridgeline_plot_for_expression_distribution_of_GSEA_result_8()
      print ("9. running score and preranked list of GSEA result")
      %R running_score_and_preranked_list_of_GSEA_result_9()
     #%R print(barplot(edo,x = "Count", color="p.adjust", showCategory=20)+ xlab("Count"))
     
      print("畫完了!!")
   else:
      print("請選擇檔案")
        

        
run_button = widgets.Button(
      description = '開始畫圖'
)

run_button.on_click(draw)
display(run_button)
output

Button(description='開始畫圖', style=ButtonStyle())

Output()